In [14]:
from __future__ import print_function
import datetime
import warnings
from math import ceil
import bs4
import MySQLdb as mdb
import requests
from pandas_datareader import data, wb
import urllib
import pandas as pd

In [2]:
def get_list_sp500():

    now = datetime.datetime.utcnow()
    
    response = requests.get(
        "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )

    soup = bs4.BeautifulSoup(response.text)
    symbolslist = soup.select("table")[0].select("tr")[1:]
    symbols = []
    for i, symbol in enumerate(symbolslist):
        tds = symbol.select("td")
        symbols.append(
        (
            tds[0].select("a")[0].text,
            "stock",
            tds[1].select("a")[0].text,
            tds[3].text,
            "USD",
            now,
            now
        ))
    return symbols

In [3]:
def insert_db(symbols, conn):

    column_str = """ticker, instrument, name, sector,
    currency, created_date, last_updated_date
    """
    
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % \
        (column_str, insert_str)

    cur = conn.cursor()
    cur.executemany(final_str, symbols)

In [4]:
def get_tickers(conn):
    cur = con.cursor()
    cur.execute("SELECT id, ticker FROM symbol")
    data = cur.fetchall()
    return [(d[0], d[1]) for d in data]

In [5]:
def get_daily_data(ticker):
    
    try:
        yf_data = data.DataReader(ticker, 'yahoo')
        prices = []
        for index, y in yf_data.iterrows():
            prices.append(
                (index,
                y['Open'], y['High'], y['Low'], y['Close'], y['Adj Close'], y['Volume'])
            )
        return prices
    except Exception as e:
        print("Could not download Yahoo data: %s" % e)
    return []

In [6]:
def insert_daily_data_into_db(
        data_vendor_id, symbol_id, daily_data, conn
    ):

    now = datetime.datetime.utcnow()

    daily_data = [
        (data_vendor_id, symbol_id, d[0], now, now,
        d[1], d[2], d[3], d[4], d[5], d[6])
        for d in daily_data
    ]

    column_str = """data_vendor_id, symbol_id, price_date, created_date,
    last_updated_date, open_price, high_price, low_price,
    close_price, volume, adj_close_price"""
    
    insert_str = ("%s, " * 11)[:-2]
    final_str = "INSERT INTO daily_price (%s) VALUES (%s)" % \
        (column_str, insert_str)

    cur = conn.cursor()
    cur.executemany(final_str, daily_data)

In [7]:
db_host = "localhost"
db_user = "sec_user"
db_pass = "kiano045"
db_name = "securities_master"

con = mdb.connect(
    host=db_host, user=db_user, passwd=db_pass, db=db_name
)

warnings.filterwarnings('ignore')

with con:
    symbols = get_list_sp500()
    insert_db(symbols, con)
    symbols = get_tickers(con)
    lentickers = len(symbols)
    for i, t in enumerate(symbols):
        print(
            "Adding data for %s: %s out of %s" %
            (t[1], i+1, lentickers)
        )
        
        sym = t[1].replace('.', '%2E')
        try:
            yf_data = get_daily_data(sym)
            insert_daily_data_into_db('1', t[0], yf_data, con)
        except Exception as e:
            print("Error processing %s" % t[1])

    print("Successfully added data.")

Adding data for MMM: 1 out of 505
Adding data for ABT: 2 out of 505
Adding data for ABBV: 3 out of 505
Adding data for ACN: 4 out of 505
Adding data for ATVI: 5 out of 505
Adding data for AYI: 6 out of 505
Adding data for ADBE: 7 out of 505
Adding data for AMD: 8 out of 505
Adding data for AAP: 9 out of 505
Adding data for AES: 10 out of 505
Adding data for AET: 11 out of 505
Adding data for AMG: 12 out of 505
Adding data for AFL: 13 out of 505
Adding data for A: 14 out of 505
Adding data for APD: 15 out of 505
Adding data for AKAM: 16 out of 505
Adding data for ALK: 17 out of 505
Adding data for ALB: 18 out of 505
Adding data for ARE: 19 out of 505
Adding data for ALXN: 20 out of 505
Adding data for ALGN: 21 out of 505
Adding data for ALLE: 22 out of 505
Adding data for AGN: 23 out of 505
Adding data for ADS: 24 out of 505
Adding data for LNT: 25 out of 505
Adding data for ALL: 26 out of 505
Adding data for GOOGL: 27 out of 505
Adding data for GOOG: 28 out of 505
Adding data for MO: 2

Adding data for GIS: 217 out of 505
Adding data for GM: 218 out of 505
Adding data for GPC: 219 out of 505
Adding data for GILD: 220 out of 505
Adding data for GPN: 221 out of 505
Adding data for GS: 222 out of 505
Adding data for GT: 223 out of 505
Adding data for GWW: 224 out of 505
Adding data for HAL: 225 out of 505
Adding data for HBI: 226 out of 505
Adding data for HOG: 227 out of 505
Adding data for HRS: 228 out of 505
Adding data for HIG: 229 out of 505
Adding data for HAS: 230 out of 505
Adding data for HCA: 231 out of 505
Adding data for HCP: 232 out of 505
Adding data for HP: 233 out of 505
Adding data for HSIC: 234 out of 505
Adding data for HSY: 235 out of 505
Adding data for HES: 236 out of 505
Adding data for HPE: 237 out of 505
Adding data for HLT: 238 out of 505
Error processing HLT
Adding data for HOLX: 239 out of 505
Adding data for HD: 240 out of 505
Adding data for HON: 241 out of 505
Adding data for HRL: 242 out of 505
Adding data for HST: 243 out of 505
Adding da

Adding data for TXN: 440 out of 505
Adding data for TXT: 441 out of 505
Adding data for TMO: 442 out of 505
Adding data for TIF: 443 out of 505
Adding data for TWX: 444 out of 505
Adding data for TJX: 445 out of 505
Adding data for TMK: 446 out of 505
Adding data for TSS: 447 out of 505
Adding data for TSCO: 448 out of 505
Adding data for TDG: 449 out of 505
Adding data for TRV: 450 out of 505
Adding data for TRIP: 451 out of 505
Adding data for FOXA: 452 out of 505
Adding data for FOX: 453 out of 505
Adding data for TSN: 454 out of 505
Adding data for UDR: 455 out of 505
Adding data for ULTA: 456 out of 505
Adding data for USB: 457 out of 505
Adding data for UA: 458 out of 505
Error processing UA
Adding data for UAA: 459 out of 505
Adding data for UNP: 460 out of 505
Adding data for UAL: 461 out of 505
Adding data for UNH: 462 out of 505
Adding data for UPS: 463 out of 505
Adding data for URI: 464 out of 505
Adding data for UTX: 465 out of 505
Adding data for UHS: 466 out of 505
Addin

In [9]:
errored = ['BHGE', 'BHF', 'CDNS', 'DWDP', 'RE', 'HLT', 'PKG', 'PFG', 'RJF', 'REG', 'SNA', 'STI', 'UA', 'ZION']

In [17]:
# Read Data
con = mdb.connect(
    host=db_host, user=db_user, passwd=db_pass, db=db_name
)

sql = """SELECT dp.price_date, dp.adj_close_price
FROM symbol AS sym
INNER JOIN daily_price AS dp
ON dp.symbol_id = sym.id
WHERE sym.ticker = 'GOOG'
ORDER BY dp.price_date ASC;"""

goog = pd.read_sql_query(sql, con=con, index_col="price_date")
print(goog.tail())

            adj_close_price
price_date                 
2017-09-18        1306900.0
2017-09-19         936700.0
2017-09-20        1669800.0
2017-09-21        1290600.0
2017-09-22        1040400.0
